<a href="https://colab.research.google.com/github/RemyaVKarthikeyan/AA-Stagecoach-Project/blob/main/18_07_2024_22_13_SWT_all_hours.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

18/07/2024

In [18]:
import pandas as pd
import requests
from difflib import SequenceMatcher

# Function to normalize stop names
def normalize_stop_name(name):
    return ' '.join(name.lower().split())

# Modify this path accordingly
file_path = '/content/QSI points.xlsx'

# Read the Excel file into a DataFrame
df = pd.read_excel(file_path)

# Get the lineID from the user
lineID = input("Please enter the lineID: ")

# Check if the necessary column is present
if 'Route_Dir_QSI_No' not in df.columns:
    print("The 'Route_Dir_QSI_No' column is not present in the provided file.")
else:
    # Convert the lineID to uppercase to ensure case-insensitivity
    lineID = lineID.upper()

    # Convert the 'Route_Dir_QSI_No' column to uppercase for comparison
    df['Route_Dir_QSI_No'] = df['Route_Dir_QSI_No'].str.upper()

    # Normalize the stop names in the DataFrame
    df['STOP_NAME'] = df['STOP_NAME'].apply(normalize_stop_name)

    # Create regular expressions for filtering
    pattern_A = f"^{lineID}_A\\d+$"  # Regular expression for lineID_A**
    pattern_B = f"^{lineID}_B\\d+$"  # Regular expression for lineID_B**

    # Filter rows where the 'Route_Dir_QSI_No' column matches the pattern
    filtered_df_A = df[df['Route_Dir_QSI_No'].str.match(pattern_A, na=False)][['Route_Dir_QSI_No', 'STOP_NAME']]
    filtered_df_B = df[df['Route_Dir_QSI_No'].str.match(pattern_B, na=False)][['Route_Dir_QSI_No', 'STOP_NAME']]

    # Function to fetch and process route sequence data from TfL API
    def fetch_and_process_route_data(route_type, pattern, filtered_df):
        api_url = f"https://api.tfl.gov.uk/Line/{lineID}/Route/Sequence/{route_type}"
        response = requests.get(api_url)

        results_list = []

        if response.status_code == 200:
            route_data = response.json()

            # Iterate through each stop in the route data
            for stop in route_data['stopPointSequences'][0]['stopPoint']:
                stop_name_api = normalize_stop_name(stop['name'])
                stop_id = stop['id']

                # Check if the stop_name_api exists in the filtered DataFrame for the correct direction
                matched_row = filtered_df[(filtered_df['STOP_NAME'] == stop_name_api) &
                                          (filtered_df['Route_Dir_QSI_No'].str.match(pattern))]

                if not matched_row.empty:
                    route_dir_qsi_no = matched_row.iloc[0]['Route_Dir_QSI_No']
                    results_list.append({
                        'Route_Dir_QSI_No': route_dir_qsi_no,
                        'STOP_Name': stop['name'],
                        'ID': stop_id
                    })
                else:
                    # If exact match not found, try partial matching based on words before and after '/'
                    api_stop_name_parts = stop_name_api.split('/')
                    for index, row in filtered_df.iterrows():
                        df_stop_name_parts = row['STOP_NAME'].split('/')
                        for api_part in api_stop_name_parts:
                            for df_part in df_stop_name_parts:
                                if SequenceMatcher(None, df_part.strip(), api_part.strip()).ratio() > 0.8:
                                    matched_row = pd.DataFrame([row])
                                    break
                            if not matched_row.empty:
                                break
                        if not matched_row.empty:
                            break

                    if not matched_row.empty:
                        route_dir_qsi_no = matched_row.iloc[0]['Route_Dir_QSI_No']
                        results_list.append({
                            'Route_Dir_QSI_No': route_dir_qsi_no,
                            'STOP_Name': stop['name'],
                            'ID': stop_id
                        })
        else:
            print(f"Failed to fetch route sequence data from TfL API for {route_type} route. Status code: {response.status_code}")

        return results_list

    # Fetch and process outbound route data for _A**
    matched_results_A = fetch_and_process_route_data('outbound', pattern_A, filtered_df_A)

    # Fetch and process inbound route data for _B**
    matched_results_B = fetch_and_process_route_data('inbound', pattern_B, filtered_df_B)

    # Create DataFrames from the matched results for each direction
    matched_results_df_A = pd.DataFrame(matched_results_A)
    matched_results_df_B = pd.DataFrame(matched_results_B)

    # Function to remove partial matches if exact matches are found
    def remove_partial_matches(exact_df, matched_df):
        for index, row in exact_df.iterrows():
            exact_stop_name = row['STOP_NAME']
            route_dir_qsi_no = row['Route_Dir_QSI_No']
            # Find exact matches in matched_df
            exact_matches = matched_df[(matched_df['Route_Dir_QSI_No'] == route_dir_qsi_no) &
                                       (matched_df['STOP_Name'].apply(normalize_stop_name) == exact_stop_name)]
            if not exact_matches.empty:
                # Remove partial matches
                matched_df = matched_df[~((matched_df['Route_Dir_QSI_No'] == route_dir_qsi_no) &
                                          (matched_df['STOP_Name'].apply(normalize_stop_name) != exact_stop_name))]
        return matched_df

    # Remove partial matches for direction A
    matched_results_df_A = remove_partial_matches(filtered_df_A, matched_results_df_A)

    # Remove partial matches for direction B
    matched_results_df_B = remove_partial_matches(filtered_df_B, matched_results_df_B)

    # Remove duplicate stop names with the same Route_Dir_QSI_No and different IDs
    matched_results_df_A = matched_results_df_A.drop_duplicates(subset=['Route_Dir_QSI_No', 'STOP_Name'], keep='first')
    matched_results_df_B = matched_results_df_B.drop_duplicates(subset=['Route_Dir_QSI_No', 'STOP_Name'], keep='first')

    # Print the matched results for direction A
    print(f"\n\n\033[1m\033[4mQSI stop points for direction {lineID}_A\033[0m\n")
    matched_results_df_A = matched_results_df_A[matched_results_df_A['Route_Dir_QSI_No'].str.match(pattern_A)]
    print(matched_results_df_A[['Route_Dir_QSI_No', 'STOP_Name', 'ID']])

    # Print the matched results for direction B
    print(f"\n\n\033[1m\033[4mQSI stop points for direction {lineID}_B\033[0m\n")
    matched_results_df_B = matched_results_df_B[matched_results_df_B['Route_Dir_QSI_No'].str.match(pattern_B)]
    print(matched_results_df_B[['Route_Dir_QSI_No', 'STOP_Name', 'ID']])

    # Concatenate the matched results DataFrames for directions A and B
    combined_df = pd.concat([matched_results_df_A, matched_results_df_B], ignore_index=True)

    # Display the combined DataFrame
    print("\n\n\033[1m\033[4mCombined QSI stop points for directions A and B\033[0m\n")
    print(combined_df[['Route_Dir_QSI_No', 'STOP_Name', 'ID']])


KeyboardInterrupt: Interrupted by user

In [19]:
import pandas as pd
import requests
from datetime import datetime
import pytz

# Function to normalize stop names
def normalize_stop_name(name):
    return ' '.join(name.lower().split())

# Function to fetch data from the API
def fetch_data(url):
    response = requests.get(url)
    return response.json()

# Function to extract schedule names
def extract_schedule_names(data, schedule_names_dict={}):
    if isinstance(data, dict):
        if data.get('$type') == "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities" and 'knownJourneys' in data:
            if 'name' in data:
                schedule_names_dict[data['name']] = data['knownJourneys']
        for key, value in data.items():
            extract_schedule_names(value, schedule_names_dict)
    elif isinstance(data, list):
        for item in data:
            extract_schedule_names(item, schedule_names_dict)
    return schedule_names_dict

# Function to categorize journeys into hourly slots
def categorize_into_slots(timetable):
    slots = [[] for _ in range(24)]
    for journey in timetable:
        hour = int(journey['hour'])  # Convert hour to integer
        if 0 <= hour < 24:  # Ensure hour is within the valid range
            slots[hour].append(journey)
    return slots

# Function to fetch the current day of the week
def get_day_of_week():
    bst = pytz.timezone('Europe/London')
    now = datetime.now(bst)
    return now.strftime('%A')  # %A gives full weekday name (e.g., 'Monday')

# Function to calculate Scheduled Wait Time (SWT) for all hours
def calculate_swt_for_all_hours(slots):
    swt_per_hour = []
    for hour in range(24):
        total_buses = len(slots[hour])
        scheduled_wait_time = 60 / (total_buses * 2) if total_buses > 0 else None  # Use None to indicate no buses
        swt_per_hour.append((scheduled_wait_time, total_buses))
    return swt_per_hour

# Function to select the preferred schedule name based on the current day of the week
def select_preferred_schedule(schedule_names_dict, day_of_week):
    if day_of_week.lower() in ['monday', 'tuesday', 'wednesday', 'thursday']:
        preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Thursday', 'Monday to Friday']
    elif day_of_week.lower() == 'friday':
        preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Friday', 'Friday']
    elif day_of_week.lower() == 'saturday':
        preferred_schedule_names = ['Saturday']
    elif day_of_week.lower() == 'sunday':
        preferred_schedule_names = ['Sunday']
    else:
        preferred_schedule_names = [day_of_week]

    for preferred_name in preferred_schedule_names:
        if preferred_name in schedule_names_dict:
            return preferred_name
    return None

# Main logic to fetch and calculate SWT
def main(combined_df, lineID):
    bst = pytz.timezone('Europe/London')
    swt_data = {
        'Route_Dir_QSI_No': [],
        'ID': [],
    }
    # Initialize keys for all 24 hours in the dictionary
    for hour in range(24):
        swt_data[f'SWT_{hour}'] = []
        swt_data[f'Sch_{hour}'] = []

    # Update current time and hour
    current_time = datetime.now(bst)
    day_of_week = get_day_of_week()

    # Store selected schedule name to ensure it's printed only once
    selected_schedule_name = None
    printed_schedule_name = False

    for index, row in combined_df.iterrows():
        stop_point_id = row['ID']
        route_dir_qsi_no = row['Route_Dir_QSI_No']

        if f"{lineID}_A" in route_dir_qsi_no:
            direction = 'outbound'
        elif f"{lineID}_B" in route_dir_qsi_no:
            direction = 'inbound'
        else:
            continue

        url = f'https://api.tfl.gov.uk/Line/{lineID}/Timetable/{stop_point_id}?direction={direction}'
        data = fetch_data(url)

        schedule_names_dict = extract_schedule_names(data)

        if not selected_schedule_name:
            selected_schedule_name = select_preferred_schedule(schedule_names_dict, day_of_week)

        if selected_schedule_name and not printed_schedule_name:
            print(f"\n\033[1m\033[4mToday is {day_of_week}. The selected Schedule name is {selected_schedule_name}.\033[0m")
            printed_schedule_name = True

        if selected_schedule_name:
            timetable = schedule_names_dict[selected_schedule_name]
            slots = categorize_into_slots(timetable)

            # Calculate SWT for all hours
            swt_per_hour = calculate_swt_for_all_hours(slots)

            # Store SWT data for all hours
            swt_data['Route_Dir_QSI_No'].append(route_dir_qsi_no)
            swt_data['ID'].append(stop_point_id)
            for hour in range(24):
                swt, total_buses = swt_per_hour[hour]
                swt_data[f'SWT_{hour}'].append(swt)
                swt_data[f'Sch_{hour}'].append(total_buses)

    # Create DataFrame for SWT data
    swt_df = pd.DataFrame(swt_data)
    #print(swt_df)
    # Print the SWT DataFrame
    #print("\n\nSWT DataFrame:")
    #print(swt_df.to_string())

# Assuming combined_df and lineID are defined and available
# combined_df = ... (from previous code)
# lineID = ... (from user input)
main(combined_df, lineID)



Today is Thursday. The selected Schedule name is Mon-Fri Schooldays.
   Route_Dir_QSI_No          ID SWT_0  Sch_0 SWT_1  Sch_1 SWT_2  Sch_2 SWT_3  Sch_3  SWT_4  Sch_4  SWT_5  Sch_5  SWT_6  Sch_6  SWT_7  Sch_7  SWT_8  Sch_8  SWT_9  Sch_9  SWT_10  Sch_10  SWT_11  Sch_11  SWT_12  Sch_12  SWT_13  Sch_13  SWT_14  Sch_14  SWT_15  Sch_15  SWT_16  Sch_16  SWT_17  Sch_17  SWT_18  Sch_18  SWT_19  Sch_19  SWT_20  Sch_20  SWT_21  Sch_21  SWT_22  Sch_22  SWT_23  Sch_23
0            179_A1  490001063C  None      0  None      0  None      0  None      0   30.0      1   10.0      3    7.5      4    6.0      5    6.0      5    6.0      5     6.0       5     6.0       5     6.0       5     6.0       5     5.0       6     6.0       5     6.0       5     6.0       5     6.0       5     7.5       4    10.0       3    10.0       3    10.0       3    10.0       3
1            179_A2  490007021E  None      0  None      0  None      0  None      0   30.0      1   10.0      3   10.0      3    5.0      6    7.5

In [20]:
import pandas as pd
import requests
from datetime import datetime
import pytz

# Function to normalize stop names
def normalize_stop_name(name):
    return ' '.join(name.lower().split())

# Function to fetch data from the API
def fetch_data(url):
    response = requests.get(url)
    return response.json()

# Function to extract schedule names
def extract_schedule_names(data, schedule_names_dict={}):
    if isinstance(data, dict):
        if data.get('$type') == "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities" and 'knownJourneys' in data:
            if 'name' in data:
                schedule_names_dict[data['name']] = data['knownJourneys']
        for key, value in data.items():
            extract_schedule_names(value, schedule_names_dict)
    elif isinstance(data, list):
        for item in data:
            extract_schedule_names(item, schedule_names_dict)
    return schedule_names_dict

# Function to categorize journeys into hourly slots
def categorize_into_slots(timetable):
    slots = [[] for _ in range(24)]
    for journey in timetable:
        hour = int(journey['hour'])  # Convert hour to integer
        if 0 <= hour < 24:  # Ensure hour is within the valid range
            slots[hour].append(journey)
    return slots

# Function to fetch the current day of the week
def get_day_of_week():
    bst = pytz.timezone('Europe/London')
    now = datetime.now(bst)
    return now.strftime('%A')  # %A gives full weekday name (e.g., 'Monday')

# Function to calculate Scheduled Wait Time (SWT) for all hours
def calculate_swt_for_all_hours(slots):
    swt_per_hour = []
    for hour in range(24):
        total_buses = len(slots[hour])
        scheduled_wait_time = 60 / (total_buses * 2) if total_buses > 0 else None  # Use None to indicate no buses
        swt_per_hour.append((scheduled_wait_time, total_buses))
    return swt_per_hour

# Function to select the preferred schedule name based on the current day of the week
def select_preferred_schedule(schedule_names_dict, day_of_week):
    if day_of_week.lower() in ['monday', 'tuesday', 'wednesday', 'thursday']:
        preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Thursday', 'Monday to Friday']
    elif day_of_week.lower() == 'friday':
        preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Friday', 'Friday']
    elif day_of_week.lower() == 'saturday':
        preferred_schedule_names = ['Saturday']
    elif day_of_week.lower() == 'sunday':
        preferred_schedule_names = ['Sunday']
    else:
        preferred_schedule_names = [day_of_week]

    for preferred_name in preferred_schedule_names:
        if preferred_name in schedule_names_dict:
            return preferred_name
    return None

# Main logic to fetch and calculate SWT
def main(combined_df, lineID):
    bst = pytz.timezone('Europe/London')
    swt_data = {
        'Route_Dir_QSI_No': [],
        'ID': [],
    }
    # Initialize keys for all 24 hours in the dictionary
    for hour in range(24):
        swt_data[f'SWT_{hour}'] = []
        swt_data[f'Sch_{hour}'] = []

    # Update current time and hour
    current_time = datetime.now(bst)
    day_of_week = get_day_of_week()

    # Store selected schedule name to ensure it's printed only once
    selected_schedule_name = None
    printed_schedule_name = False

    for index, row in combined_df.iterrows():
        stop_point_id = row['ID']
        route_dir_qsi_no = row['Route_Dir_QSI_No']

        if f"{lineID}_A" in route_dir_qsi_no:
            direction = 'outbound'
        elif f"{lineID}_B" in route_dir_qsi_no:
            direction = 'inbound'
        else:
            continue

        url = f'https://api.tfl.gov.uk/Line/{lineID}/Timetable/{stop_point_id}?direction={direction}'
        data = fetch_data(url)

        schedule_names_dict = extract_schedule_names(data)

        if not selected_schedule_name:
            selected_schedule_name = select_preferred_schedule(schedule_names_dict, day_of_week)

        if selected_schedule_name and not printed_schedule_name:
            print(f"\n\033[1m\033[4mToday is {day_of_week}. The selected Schedule name is {selected_schedule_name}.\033[0m")
            printed_schedule_name = True

        if selected_schedule_name:
            timetable = schedule_names_dict[selected_schedule_name]
            slots = categorize_into_slots(timetable)

            # Calculate SWT for all hours
            swt_per_hour = calculate_swt_for_all_hours(slots)

            # Store SWT data for all hours
            swt_data['Route_Dir_QSI_No'].append(route_dir_qsi_no)
            swt_data['ID'].append(stop_point_id)
            for hour in range(24):
                swt, total_buses = swt_per_hour[hour]
                swt_data[f'SWT_{hour}'].append(swt)
                swt_data[f'Sch_{hour}'].append(total_buses)

    # Create DataFrame for SWT data
    swt_df = pd.DataFrame(swt_data)

    # Calculate Route SWT for each hour for directions A and B
    route_swt_data = {
        'Hour': [],
        'Route SWT A': [],
        'Route SWT B': []
    }

    for hour in range(24):
        # Calculate Route SWT for direction A
        swt_a = swt_df[swt_df['Route_Dir_QSI_No'].str.contains(f'{lineID}_A')]
        weighted_sum_a = sum(swt_a[f'SWT_{hour}'] * swt_a[f'Sch_{hour}']) if not swt_a.empty else 0
        total_buses_a = sum(swt_a[f'Sch_{hour}']) if not swt_a.empty else 0
        route_swt_a = weighted_sum_a / total_buses_a if total_buses_a > 0 else None

        # Calculate Route SWT for direction B
        swt_b = swt_df[swt_df['Route_Dir_QSI_No'].str.contains(f'{lineID}_B')]
        weighted_sum_b = sum(swt_b[f'SWT_{hour}'] * swt_b[f'Sch_{hour}']) if not swt_b.empty else 0
        total_buses_b = sum(swt_b[f'Sch_{hour}']) if not swt_b.empty else 0
        route_swt_b = weighted_sum_b / total_buses_b if total_buses_b > 0 else None

        route_swt_data['Hour'].append(hour)
        route_swt_data['Route SWT A'].append(route_swt_a)
        route_swt_data['Route SWT B'].append(route_swt_b)

    route_swt_df = pd.DataFrame(route_swt_data)

    # Print the Route SWT DataFrame
    print("\n\nRoute SWT DataFrame:")
    print(route_swt_df.to_string())

# Assuming combined_df and lineID are defined and available
# combined_df = ... (from previous code)
# lineID = ... (from user input)
main(combined_df, lineID)



Today is Thursday. The selected Schedule name is Mon-Fri Schooldays.


Route SWT DataFrame:
    Hour  Route SWT A  Route SWT B
0      0          NaN          NaN
1      1          NaN          NaN
2      2          NaN          NaN
3      3          NaN          NaN
4      4          NaN          NaN
5      5    10.500000    14.000000
6      6     9.545455     8.750000
7      7     6.774194     5.833333
8      8     6.363636     5.526316
9      9     5.384615     6.000000
10    10     5.833333     6.000000
11    11     6.000000     6.000000
12    12     6.000000     6.000000
13    13     6.000000     6.000000
14    14     5.675676     5.675676
15    15     6.000000     6.000000
16    16     5.833333     6.176471
17    17     6.000000     6.000000
18    18     5.675676     6.000000
19    19     6.000000     5.833333
20    20     8.750000     8.400000
21    21     9.545455    10.000000
22    22    10.000000    10.000000
23    23     9.545455     9.545455


In [22]:
import pandas as pd
import requests
from datetime import datetime
import pytz

# Function to normalize stop names
def normalize_stop_name(name):
    return ' '.join(name.lower().split())

# Function to fetch data from the API
def fetch_data(url):
    response = requests.get(url)
    return response.json()

# Function to extract schedule names
def extract_schedule_names(data, schedule_names_dict={}):
    if isinstance(data, dict):
        if data.get('$type') == "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities" and 'knownJourneys' in data:
            if 'name' in data:
                schedule_names_dict[data['name']] = data['knownJourneys']
        for key, value in data.items():
            extract_schedule_names(value, schedule_names_dict)
    elif isinstance(data, list):
        for item in data:
            extract_schedule_names(item, schedule_names_dict)
    return schedule_names_dict

# Function to categorize journeys into hourly slots
def categorize_into_slots(timetable):
    slots = [[] for _ in range(24)]
    for journey in timetable:
        hour = int(journey['hour'])  # Convert hour to integer
        if 0 <= hour < 24:  # Ensure hour is within the valid range
            slots[hour].append(journey)
    return slots

# Function to fetch the current day of the week
def get_day_of_week():
    bst = pytz.timezone('Europe/London')
    now = datetime.now(bst)
    return now.strftime('%A')  # %A gives full weekday name (e.g., 'Monday')

# Function to calculate Scheduled Wait Time (SWT) for all hours
def calculate_swt_for_all_hours(slots):
    swt_per_hour = []
    for hour in range(24):
        total_buses = len(slots[hour])
        scheduled_wait_time = 60 / (total_buses * 2) if total_buses > 0 else None  # Use None to indicate no buses
        swt_per_hour.append((scheduled_wait_time, total_buses))
    return swt_per_hour

# Function to select the preferred schedule name based on the current day of the week
def select_preferred_schedule(schedule_names_dict, day_of_week):
    if day_of_week.lower() in ['monday', 'tuesday', 'wednesday', 'thursday']:
        preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Thursday', 'Monday to Friday']
    elif day_of_week.lower() == 'friday':
        preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Friday', 'Friday']
    elif day_of_week.lower() == 'saturday':
        preferred_schedule_names = ['Saturday']
    elif day_of_week.lower() == 'sunday':
        preferred_schedule_names = ['Sunday']
    else:
        preferred_schedule_names = [day_of_week]

    for preferred_name in preferred_schedule_names:
        if preferred_name in schedule_names_dict:
            return preferred_name
    return None

# Main logic to fetch and calculate SWT
def main(combined_df, lineID):
    bst = pytz.timezone('Europe/London')
    swt_data = {
        'Route_Dir_QSI_No': [],
        'ID': [],
    }
    # Initialize keys for all 24 hours in the dictionary
    for hour in range(24):
        swt_data[f'SWT_{hour}'] = []
        swt_data[f'Sch_{hour}'] = []

    # Update current time and hour
    current_time = datetime.now(bst)
    day_of_week = get_day_of_week()

    # Store selected schedule name to ensure it's printed only once
    selected_schedule_name = None
    printed_schedule_name = False

    for index, row in combined_df.iterrows():
        stop_point_id = row['ID']
        route_dir_qsi_no = row['Route_Dir_QSI_No']

        if f"{lineID}_A" in route_dir_qsi_no:
            direction = 'outbound'
        elif f"{lineID}_B" in route_dir_qsi_no:
            direction = 'inbound'
        else:
            continue

        url = f'https://api.tfl.gov.uk/Line/{lineID}/Timetable/{stop_point_id}?direction={direction}'
        data = fetch_data(url)

        schedule_names_dict = extract_schedule_names(data)

        if not selected_schedule_name:
            selected_schedule_name = select_preferred_schedule(schedule_names_dict, day_of_week)

        if selected_schedule_name and not printed_schedule_name:
            print(f"\n\033[1m\033[4mToday is {day_of_week}. The selected Schedule name is {selected_schedule_name}.\033[0m")
            printed_schedule_name = True

        if selected_schedule_name:
            timetable = schedule_names_dict[selected_schedule_name]
            slots = categorize_into_slots(timetable)

            # Calculate SWT for all hours
            swt_per_hour = calculate_swt_for_all_hours(slots)

            # Store SWT data for all hours
            swt_data['Route_Dir_QSI_No'].append(route_dir_qsi_no)
            swt_data['ID'].append(stop_point_id)
            for hour in range(24):
                swt, total_buses = swt_per_hour[hour]
                swt_data[f'SWT_{hour}'].append(swt)
                swt_data[f'Sch_{hour}'].append(total_buses)

    # Create DataFrame for SWT data
    swt_df = pd.DataFrame(swt_data)

    # Calculate Route SWT for each hour for directions A and B
    route_swt_data = {
        'Hour': [],
        'Route SWT A': [],
        'Route SWT B': []
    }

    for hour in range(24):
        # Calculate Route SWT for direction A
        swt_a = swt_df[swt_df['Route_Dir_QSI_No'].str.contains(f'{lineID}_A')]
        valid_swt_a = swt_a[pd.notna(swt_a[f'SWT_{hour}'])]  # Filter out NaN values
        weighted_sum_a = sum(valid_swt_a[f'SWT_{hour}'] * valid_swt_a[f'Sch_{hour}']) if not valid_swt_a.empty else 0
        total_buses_a = sum(valid_swt_a[f'Sch_{hour}']) if not valid_swt_a.empty else 0
        route_swt_a = round(weighted_sum_a / total_buses_a, 2) if total_buses_a > 0 else None


        # Calculate Route SWT for direction B
        swt_b = swt_df[swt_df['Route_Dir_QSI_No'].str.contains(f'{lineID}_B')]
        valid_swt_b = swt_b[pd.notna(swt_b[f'SWT_{hour}'])]  # Filter out NaN values
        weighted_sum_b = sum(valid_swt_b[f'SWT_{hour}'] * valid_swt_b[f'Sch_{hour}']) if not valid_swt_b.empty else 0
        total_buses_b = sum(valid_swt_b[f'Sch_{hour}']) if not valid_swt_b.empty else 0
        route_swt_b = round(weighted_sum_b / total_buses_b, 2) if total_buses_b > 0 else None

        route_swt_data['Hour'].append(hour)
        route_swt_data['Route SWT A'].append(route_swt_a)
        route_swt_data['Route SWT B'].append(route_swt_b)

    route_swt_df = pd.DataFrame(route_swt_data)

    # Print the Route SWT DataFrame
    print("\n\nRoute SWT DataFrame:")
    print(route_swt_df.to_string())

# Assuming combined_df and lineID are defined and available
# combined_df = ... (from previous code)
# lineID = ... (from user input)
main(combined_df, lineID)



Today is Thursday. The selected Schedule name is Mon-Fri Schooldays.


Route SWT DataFrame:
    Hour  Route SWT A  Route SWT B
0      0          NaN          NaN
1      1          NaN          NaN
2      2          NaN          NaN
3      3          NaN          NaN
4      4        30.00          NaN
5      5        10.50        14.00
6      6         9.55         8.75
7      7         6.77         5.83
8      8         6.36         5.53
9      9         5.38         6.00
10    10         5.83         6.00
11    11         6.00         6.00
12    12         6.00         6.00
13    13         6.00         6.00
14    14         5.68         5.68
15    15         6.00         6.00
16    16         5.83         6.18
17    17         6.00         6.00
18    18         5.68         6.00
19    19         6.00         5.83
20    20         8.75         8.40
21    21         9.55        10.00
22    22        10.00        10.00
23    23         9.55         9.55
